# Political parties

We want to add a political party affiliation for each jurisdiction in landvote.

To do so, we match each jurisdiction's election results for the most recent presidential election and assign it the party of the majority candidate. Parties can change over time, so we calculate it based on the **popular vote** of the most recent presidential election from the measure date.

Example: DeKalb County, GA passed a measure on 2001-03-20 and the party is Democrat. We assigned this party based on the candidate from the 2000 presidential election that had the most votes in DeKalb County, which was Al Gore. 

The preprocessing scripts for the census data can be found in the `datasets/political_parties` directory.  

In [ ]:
import ibis
from ibis import _
import ibis.expr.datatypes as dt  
import re
from cng.utils import *
from cng.h3 import *
from minio import Minio

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial"])
set_secrets(con)



In [ ]:
landvote = (con.read_parquet('s3://public-tpl/landvote/landvote_geom.parquet')
            .mutate(election_year =  _.year - _.year%4) # most recent presidential election year, used to set party 
           )
state_parties = con.read_parquet('s3://public-election/state/state_political_parties_1976-2024.parquet').rename(election_year='year')
county_parties = con.read_parquet('s3://public-election/county/county_political_parties_1988-2024.parquet').rename(election_year='year')


## Joining landvote with political parties

In [ ]:
landvote_states = landvote.filter(_.jurisdiction=="State")
landvote_other = landvote.filter(_.jurisdiction!="State")

states = landvote_states.join(state_parties, ['state','election_year'], how='left').drop('state_right','election_year_right')
other = landvote_other.join(county_parties, ['state','county','election_year'], how='left').drop('FIPS','state_right','county_right','election_year_right')

landvote_parties=states.union(other).drop('geom_right')
save_url = 's3://public-tpl/landvote/landvote_party.parquet'
landvote_parties.to_parquet(save_url)

#### Checking which landvote entries didn't match with the party data 

Reasons why the party is missing 
- Broomfield, CO wasn't a county until 2001
- Alaska doesn't report county-level election results
- Roanoke city and Fairfax city don't record their election results prior to 2020

In [ ]:
landvote_parties.filter(_.party.isnull()).order_by('year','state').execute()

## Save to pmtiles

In [ ]:
geobuf_file = 'landvote_party.fgb'
landvote_parties.execute().to_file(geobuf_file)
pmtiles_file = 'landvote_party.pmtiles'
pmtiles = f"s3://public-tpl/landvote/{pmtiles_file}"
source_layer_name = re.sub(r'\W+', '', os.path.splitext(os.path.basename(pmtiles_file))[0])

options =[f'--layer={source_layer_name}',
            '--extend-zooms-if-still-dropping',  
         ]
new_pmtiles = to_pmtiles(geobuf_file, pmtiles_file, options = options )
s3_cp(new_pmtiles,pmtiles)